### [요구사항 1] titanic 딥러닝 모델 기본 훈련

In [ ]:
import torch
import sys
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse

# 경로 설정 및 데이터셋 로드
# Titanic 데이터셋 이진 분류 문제
# BASE_PATH를 sys.path에 추가하여 커스텀 모듈 import 가능

BASE_PATH = "/Users/leejunyoung/Documents/3-2/deep_Learning/practice/link_dl"
if BASE_PATH not in sys.path:
    sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset

# DataLoader 생성 함수
# 훈련과 검증 데이터셋을 불러옴
# batch_size는 wandb 설정에서 받아옴

def get_data():
  titanic_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(titanic_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=titanic_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

  return train_data_loader, validation_data_loader

# 모델 정의
# 입력 : 10개 feature (전처리된 Titanic 데이터셋)
# 출력 : 2개 클래스 (사망 / 생존)
# 활성화 함수 : ReLU

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델 및 옵티마이저 생성
# optimizer : SGD
# loss_fn은 training_loop 내에서 정의하고 있음
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

# 학습 루프
# 손실 함수 : CrossEntropyLoss : 타이타닉의 경우 생존, 사망 두가지로 분류되므로 분류 모델 사용
# wandb로 학습 과정 로깅
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']

      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']

        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    #wandb 로깅
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

# main 함수
# wandb, experiment 초기화
# 데이터, 모델, 학습 루프 실행

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  in_notebook = "ipykernel" in sys.modules

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=in_notebook, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args, _ = parser.parse_known_args()

  main(args)

Namespace(wandb=True, batch_size=512, epochs=1000)
{'epochs': 1000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11

Epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
Training loss,█▆▇▄▅▄▄▃▂▃▃▃▂▂▄▂▃▃▃▃▂▂▂▄▃▃▄▁▂▂▂▃▁▃▃▂▄▄▃▁
Validation loss,█▆▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.58417
Validation loss,0.52056


##### 기술적 사항
- 데이터셋 : Titanic
  - 타깃 값(survived)이 0 또는 1인 이진 분류 문제
  - 결측값 처리 및 범주형으로 전처리 완료
- 모델 구조
  - 입력 차원: 10
  - 은닉층: 20 -> 20 (ReLU 활성화)
  - 출력 차원 : 2 (사망/생존)
- 손실 함수 : CrossEntropyLoss
  - 확률 기반 분류 문제에 적합
- Optimizer : SGD
- Batch Size: 512
- Epochs : 1000

##### 고찰
- CrossEntropyLoss와 MSELoss 차이
  - MSELoss는 회귀(연속값)에 적합, CrossEntropyLoss는 분류(클래스)에 적합
  - CrossEntropyLoss는 클래스 확률을 직접 최적화하기 때문에 분류 문제에서 더 빠르고 안정적인 수렴을 보임

- optimize의 역할
  - SGD는 간단하면서도 안정적인 기본 옵티마이저임
  - 이후 성능 향상을 위해 Adam으로 변경해볼 수 있음

- 학습률의 중요성
  - 너무 높으면 발산
  - 너무 낮으면 수렴이 느려짐
  - 현재는 0.001로 안정적인 학습이 이루어짐

##### 결과 해석
- Training Loss, Validation Loss 차이
  - 두 값이 큰 차이 없이 비슷하게 감소하고 있음
    - 과적합 징후가 거의 없음을 확인 가능
    - 현재 학습률 설정이 안정적임

- 700 epoch 이후 손실 감소 폭이 작아짐
  - Validation loss가 0.52 근처에서 수렴
  - 현재 설정으로는 모델이 더 이상 성능 향상을 크게 이루지 못하는 상태

- Validation Loss가 Train Loss보다 낮거나 거의 같은 구간 존재
  - 흔히 regularization이 잘 된 경우 이런 현상이 발생함

### [요구사항 2] Activation Function과 Batch Size 변경 및 선택하기

In [ ]:
import torch
import sys
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse

# 경로 설정 및 데이터셋 로드
# Titanic 데이터셋 이진 분류 문제
# BASE_PATH를 sys.path에 추가하여 커스텀 모듈 import 가능

BASE_PATH = "/Users/leejunyoung/Documents/3-2/deep_Learning/practice/link_dl"
if BASE_PATH not in sys.path:
    sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset

# 활성 함수를 변경하기 위한 함수
def get_activation(name: str):
    name = name.lower()
    if name == "relu":
        return nn.ReLU()
    if name == "sigmoid":
        return nn.Sigmoid()
    if name == "elu":
        return nn.ELU()
    if name in ("leakyrelu", "leaky_relu", "lrelu"):
        return nn.LeakyReLU(negative_slope=0.01)
    raise ValueError(f"Unknown activation: {name}")

# DataLoader 생성 함수
# 훈련과 검증 데이터셋을 불러옴
# batch_size는 wandb 설정에서 받아옴

def get_data():
  titanic_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(titanic_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=titanic_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

  return train_data_loader, validation_data_loader

# 모델 정의
# 입력 : 10개 feature (전처리된 Titanic 데이터셋)
# 출력 : 2개 클래스 (사망 / 생존)
# 활성화 함수 : wandb.config.activation 사용

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()
    act = get_activation(wandb.config.activation)

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      act,
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      act,
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델 및 옵티마이저 생성
# optimizer : SGD
# loss_fn은 training_loop 내에서 정의하고 있음
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

# 학습 루프
# 손실 함수 : CrossEntropyLoss : 타이타닉의 경우 생존, 사망 두가지로 분류되므로 분류 모델 사용
# wandb로 학습 과정 로깅
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']

      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']

        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    # wandb 로깅 (+ 비교용 메타 추가)
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations,
      "hparams/activation": wandb.config.activation,   # ← Facet/Filter/Color 용
      "hparams/batch_size": wandb.config.batch_size,   # ← Facet/Filter/Color 용
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

# main 함수
# wandb, experiment 초기화
# 데이터, 모델, 학습 루프 실행

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation': args.activation,     # ★ 추가: 모델이 참조하는 활성화 함수
    'group': 'act_vs_batch',           # ★ 추가: 한 프레임에 묶어보기 위한 그룹명
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="Activation x Batch sweep (Titanic)",
    tags=["my_model", "titanic"],
    name=f"{current_time_str}-act={args.activation}-bs={args.batch_size}",  # ★ 러닝 구분 쉬움
    group=config['group'],                                                 # ★ 같은 그룹으로 묶기
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  in_notebook = "ipykernel" in sys.modules

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=in_notebook, help="True or False"
  )

  parser.add_argument(
    "--activation", type=str, default="relu",
    choices=["relu", "sigmoid", "elu", "leakyrelu"], help="Hidden activation function"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args, _ = parser.parse_known_args()

  main(args)


activations = ["relu", "sigmoid", "elu", "leakyrelu"]
batch_sizes = [16, 32, 64, 128]

for act in activations:
    for bs in batch_sizes:
        wandb.init(
            mode="online",
            project="my_model_training",
            group="act_vs_batch",
            name=f"act={act}-bs={bs}",
            config={
                "epochs": 1000,
                "batch_size": bs,
                "learning_rate": 1e-3,
                "n_hidden_unit_list": [20, 20],
                "activation": act,
                "group": "act_vs_batch",
            }
        )

        # 이 아래는 실험 실행
        train_loader, val_loader = get_data()
        model, optimizer = get_model_and_optimizer()
        training_loop(model, optimizer, train_loader, val_loader)

        # wandb 세션 종료
        wandb.finish()


/opt/anaconda3/envs/link_dl/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/anaconda3/envs/link_dl/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annot

Namespace(wandb=True, activation='relu', batch_size=512, epochs=1000)
{'epochs': 1000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'activation': 'relu', 'group': 'act_vs_batch'}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750   

Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇█
Training loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▁▁▁▂▂▁▂▂▁▂▁▁▂▁▂▁▁▂▁▂
Validation loss,█▆▄▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.59998
Validation loss,0.61197
hparams/activation,relu
hparams/batch_size,512


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▂▃▃▃▃▃▄▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█████
Training loss,█▇▇▇▇▆▆▆▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁
Validation loss,██▇▇▇▆▆▆▇▅▅▆▄▄▄▃▃▃▃▃▆▂▃▃▃▇▂▃▂▂▃▂▁▄▂▃▂▃▁▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.41807
Validation loss,0.4704
hparams/activation,relu
hparams/batch_size,16


...

Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇█████
Training loss,███▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▃▃▃▂▂▁▂▂▂▂▁▁▁▂▁
Validation loss,▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▃█▃▂▃▃▁▂▁▅▅▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.41441
Validation loss,0.50742
hparams/activation,leakyrelu
hparams/batch_size,32


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇███
Training loss,█▆▅▅▅▅▆▅▅▆▅▃▄▅▅▄▅▆▄▅▆▄▄▄▄▃▃▃▄▃▃▂▂▃▁▄▂▂▃▄
Validation loss,▇██▇▇▆▆▆▆▆▆▆▆▆▅▆▅▅▄▅▅▅▄▅▄▅▄▄▄▄▃▃▂▂▄▄▁▃▃▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.53931
Validation loss,0.59921
hparams/activation,leakyrelu
hparams/batch_size,64


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇██
Training loss,█▇▆▇▆▇▆▆▅▅▅▅▅▄▅▅▅▅▄▄▄▄▅▄▄▄▄▃▃▃▃▃▃▂▃▂▂▂▁▁
Validation loss,█▇▇▆▆▆▅▅▆▅▅▅▆▅▄▅▆▄▅▄▄▄▅▅▃▄▆▃▃▄▃▃▂▃▂▂▂▁▂▂
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.54191
Validation loss,0.5766
hparams/activation,leakyrelu
hparams/batch_size,128


##### 결과 분석
- 제일 좋은 성능을 보인 Activation Function과 Batch Size 조합은 LeakyReLU + Batch size 16
  - Validation loss가 0.3909로 가장 높은 성능을 보임

- 그 외 상위권
  - ReLU, 64 : 0.4605
  - ReLU, 16 : 0.4704
  - ELU, 16 : 0.4718
  - ReLU, 32 : 0.4872

- 평균 경향
  - Batch size : 대체로 작을 수록 일반화 성능이 좋았음
  - Activation : 최고점은 LeakyReLU 였지만 평균적으로는 ReLU가 가장 안정적
  - Sigmoid: 전반적으로 가장 약한 성능을 보였음

- LeakyReLU의 강점
  - ReLU는 음수에서 기울기가 0이라 죽은 뉴런 문제 발생가능
  - LeakyReLU는 음수 영역에 작은 기울기를 주어 표현력/수렴 안정성이 개선됨
  - 특히 작은 배치 사이즈와 결합하면 노이즈가 적당한 정규화 효과를 내어 일반화에 유리하게 작용

- 배치 사이즈의 역할
  - 작은 배치는 Gradient에 노이즈 도입 따라서 과적합 억제 + 더 넓은 영역 탐색가능
  - 큰 배치는 Gradient 추정이 안정적이지만, Sharp minima로 수렴하거나 일반화가 상대적으로 나쁠 수 있음

### [요구사항 3] 테스트 및 submission.csv 생성

In [4]:
import torch
import sys
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse

# 경로 설정 및 데이터셋 로드
# Titanic 데이터셋 이진 분류 문제
# BASE_PATH를 sys.path에 추가하여 커스텀 모듈 import 가능

BASE_PATH = "/Users/leejunyoung/Documents/3-2/deep_Learning/practice/link_dl"
if BASE_PATH not in sys.path:
    sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset

# DataLoader 생성 함수
# 훈련과 검증 데이터셋을 불러옴
# batch_size는 wandb 설정에서 받아옴

def get_data():
  titanic_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(titanic_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=titanic_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

  return train_data_loader, validation_data_loader, test_data_loader

# 모델 정의
# 입력 : 10개 feature (전처리된 Titanic 데이터셋)
# 출력 : 2개 클래스 (사망 / 생존)
# 활성화 함수 : wandb.config.activation 사용

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.LeakyReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.LeakyReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델 및 옵티마이저 생성
# optimizer : SGD
# loss_fn은 training_loop 내에서 정의하고 있음
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

# 학습 루프
# 손실 함수 : CrossEntropyLoss : 타이타닉의 경우 생존, 사망 두가지로 분류되므로 분류 모델 사용
# wandb로 학습 과정 로깅
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']

      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']

        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    # wandb 로깅 (+ 비교용 메타 추가)
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations,
      "hparams/activation": wandb.config.activation,   # ← Facet/Filter/Color 용
      "hparams/batch_size": wandb.config.batch_size,   # ← Facet/Filter/Color 용
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

# main 함수
# wandb, experiment 초기화
# 데이터, 모델, 학습 루프 실행

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation': args.activation,     # ★ 추가: 모델이 참조하는 활성화 함수
    'group': 'act_vs_batch',           # ★ 추가: 한 프레임에 묶어보기 위한 그룹명
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="Activation x Batch sweep (Titanic)",
    tags=["my_model", "titanic"],
    name=f"{current_time_str}-act={args.activation}-bs={args.batch_size}",  # ★ 러닝 구분 쉬움
    group=config['group'],                                                 # ★ 같은 그룹으로 묶기
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, test_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  in_notebook = "ipykernel" in sys.modules

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=in_notebook, help="True or False"
  )

  parser.add_argument(
    "--activation", type=str, default="leakyrelu",
    choices=["relu", "sigmoid", "elu", "leakyrelu"], help="Hidden activation function"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args, _ = parser.parse_known_args()

  main(args)




Namespace(wandb=True, activation='leakyrelu', batch_size=16, epochs=1000)
{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'activation': 'leakyrelu', 'group': 'act_vs_batch'}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21

Epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
Training loss,██▇▆▆▆▆▅▅▅▄▅▄▄▄▄▃▃▂▃▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▁
Validation loss,█▇▇▇▇▆▅▆▅▅▅▄▄▄▃▆▃█▃▂▁▂▂▃▁▁▄▂▁▅▃▁▁▄▁▂▂▅▂▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.43417
Validation loss,0.43691
hparams/activation,leakyrelu
hparams/batch_size,16


In [7]:
import torch
import sys
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse

# 경로 설정 및 데이터셋 로드
# Titanic 데이터셋 이진 분류 문제
# BASE_PATH를 sys.path에 추가하여 커스텀 모듈 import 가능

BASE_PATH = "/Users/leejunyoung/Documents/3-2/deep_Learning/practice/link_dl"
if BASE_PATH not in sys.path:
    sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset

# DataLoader 생성 함수
# 훈련과 검증 데이터셋을 불러옴
# batch_size는 wandb 설정에서 받아옴

def get_data():
  titanic_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(titanic_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=titanic_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

  return train_data_loader, validation_data_loader, test_data_loader

# 모델 정의
# 입력 : 10개 feature (전처리된 Titanic 데이터셋)
# 출력 : 2개 클래스 (사망 / 생존)
# 활성화 함수 : wandb.config.activation 사용

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.LeakyReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.LeakyReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델 및 옵티마이저 생성
# optimizer : SGD
# loss_fn은 training_loop 내에서 정의하고 있음
def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer

# 학습 루프
# 손실 함수 : CrossEntropyLoss : 타이타닉의 경우 생존, 사망 두가지로 분류되므로 분류 모델 사용
# wandb로 학습 과정 로깅
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 100

  best_val_loss = float("inf")
  best_epoch = 0
  best_path = "best_model.pt"

  for epoch in range(1, n_epochs + 1):
    # -------- Train --------
    model.train()
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target']

      output_train = model(input)
      loss = loss_fn(output_train, target)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      num_trains += 1

    # -------- Validate --------
    model.eval()
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input = validation_batch['input']
        target = validation_batch['target']

        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    train_loss_avg = loss_train / max(1, num_trains)
    val_loss_avg = loss_validation / max(1, num_validations)

    # wandb 로깅
    wandb.log({
      "Epoch": epoch,
      "Training loss": train_loss_avg,
      "Validation loss": val_loss_avg,
      "hparams/activation": wandb.config.activation,
      "hparams/batch_size": wandb.config.batch_size,
    })

    # 콘솔 출력
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {train_loss_avg:.4f}, "
        f"Validation loss {val_loss_avg:.4f}"
      )
      next_print_epoch += 100

    # -------- Best 체크 & 저장 --------
    if val_loss_avg < best_val_loss:
      best_val_loss = val_loss_avg
      best_epoch = epoch
      torch.save(model.state_dict(), best_path)

  print(f"[Best] epoch={best_epoch}, val_loss={best_val_loss:.4f} -> saved: {best_path}")
  wandb.summary["best_epoch"] = best_epoch
  wandb.summary["best_val_loss"] = best_val_loss
  wandb.summary["best_model_path"] = best_path

  return best_epoch, best_val_loss, best_path


def predict_on_test(model, test_data_loader):
  model.eval()
  preds = []
  with torch.no_grad():
    for batch in test_data_loader:
      x = batch["input"]  # 테스트셋은 target이 없다고 가정
      logits = model(x)
      pred = logits.argmax(dim=1)
      preds.extend(pred.tolist())
  return preds

def write_submission(preds, passenger_ids=None, path="submission.csv"):
  if passenger_ids is None:
    # 캐글 Titanic 기본 테스트셋 ID 범위(892~1309) 가정
    passenger_ids = list(range(892, 892 + len(preds)))
  df = pd.DataFrame({"PassengerId": passenger_ids, "Survived": preds})
  df.to_csv(path, index=False)
  print(f"✅ Saved: {path} (rows={len(df)})")
  return path

# main 함수
# wandb, experiment 초기화
# 데이터, 모델, 학습 루프 실행

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation': args.activation,
    'group': 'act_vs_batch',
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="Activation x Batch sweep (Titanic)",
    tags=["my_model", "titanic"],
    name=f"{current_time_str}-act={args.activation}-bs={args.batch_size}",
    group=config['group'],
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, test_data_loader = get_data()
  model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  # ---- 학습 & 베스트 모델 저장 ----
  best_epoch, best_val_loss, best_path = training_loop(
    model=model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )

  # ---- 베스트 모델 로드 후 테스트 예측 ----
  model.load_state_dict(torch.load(best_path, map_location="cpu"))
  preds = predict_on_test(model, test_data_loader)

  # ---- submission.csv 생성 ----
  sub_path = write_submission(preds, path="submission.csv")
  wandb.save(sub_path)

  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  in_notebook = "ipykernel" in sys.modules

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=in_notebook, help="True or False"
  )

  parser.add_argument(
    "--activation", type=str, default="leakyrelu",
    choices=["relu", "sigmoid", "elu", "leakyrelu"], help="Hidden activation function"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args, _ = parser.parse_known_args()

  main(args)



Namespace(wandb=True, activation='leakyrelu', batch_size=16, epochs=1000)
{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'activation': 'leakyrelu', 'group': 'act_vs_batch'}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21

Epoch,▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Training loss,█▇▇▇▆▆▅▅▅▅▅▄▄▄▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▂▂
Validation loss,▅▅▅▅▄▄▅▅▄▅▅▄▄▄▃▆▄▂▃▂▂▇▁▂█▃▂▃▂▁▁▄▁▂▃▂▂▂▁▁
hparams/batch_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.43874
Validation loss,0.39889
best_epoch,961
best_model_path,best_model.pt
best_val_loss,0.37127
hparams/activation,leakyrelu


##### 기술적 고찰
- 모델 학습 및 검증 손실 추적
  - 1000 epoch 동안 학습을 진행하면서, 각 epoch마다 train loss, validation loss를 확인했다
  - 이과정에서 validation loss가 점진적으로 감소하다가 특정 시점이후 다시 증가하는 모습을 보였다
  - 이를 통해 과적합이 이루어지고 있다는 것을 확인할 수 있었다

- 최적 epoch 탐색
  - validation loss가 가장 낮은 epoch를 best_epoch로 지정하고 해당 시점의 모델 가중치를 best_model.pt로 저장했다

- Early Stopping 기법을 사용했다면 학습 시간을 단축하면서 동일한 결과를 얻을 수 있었을 것으로 보인다

### [요구사항 4] submission.csv 제출 및 등수확인

![img](https://drive.google.com/thumbnail?id=1FLDAm-hIx0b_szMkLtfqAHbn86MfFoQy&sz=w1200)


##### 후기

이번 과제를 통해 단순히 모델을 학습시키는 것에서 그치지 않고, 모델의 성능을 객관적으로 평가하고 분석하는 과정의 중요성을 배울 수 있었다. 특히 Validation Loss를 지속적으로 관찰하며 최적의 학습 시점을 스스로 판단하는 경험은 매우 의미 있었다. 학습 초반에는 손실이 점진적으로 감소하다가 일정 시점 이후 다시 증가하는 전형적인 과적합 패턴을 직접 확인할 수 있었고, 이를 통해 단순한 반복 학습만으로는 좋은 결과를 얻을 수 없다는 점을 깨달았다. 또한 최적의 epoch에서 모델 파라미터를 저장하고 이를 기반으로 테스트셋 예측을 진행함으로써 실험의 효율성과 재현성을 확보할 수 있었다.

submission.csv를 직접 생성해 Titanic - Machine Learning from Disaster 리더보드에 제출해본 과정은 이론적인 지식을 실제 실습으로 연결하는 경험이었다. 단순한 신경망 구조임에도 불구하고 활성화 함수나 배치 사이즈 같은 작은 설정 변화만으로도 성능에 차이가 발생한다는 점이 특히 흥미로웠다. 결과적으로 이번 과제는 모델의 성능 향상뿐 아니라, 실험을 설계하고 관리하는 과정 또한 데이터 과학에서 매우 중요한 요소임을 체감하게 해주었다. 이런 과정을 통해 단순한 코드 작성에서 한 단계 나아가, 실제 문제 해결을 위한 전략적 접근을 익힐 수 있었다.